In [11]:
def CVaR_optimization(scenario_returns, alpha=0.95):
    S, N = scenario_returns.shape
    # Create the model
    model = gp.Model("CVaR_Bond_Optimization")

    model.setParam('OutputFlag', 0)

    # Decision variables
    w = model.addVars(N, vtype=GRB.CONTINUOUS, lb=0, name="w")  # weights for bonds
    x = model.addVars(N, vtype=GRB.BINARY, name="x")  # binary selection of bonds
    VaR = model.addVar(vtype=GRB.CONTINUOUS, name="VaR")  # Value at Risk variable
    z = model.addVars(N, vtype=GRB.CONTINUOUS, lb=0, name="z")  # Auxiliary for CVaR

    # Constrain the total weight
    model.addConstr(quicksum(w[i] for i in range(N)) == 1, "WeightSum")

    # Deviation from benchmark
    deviation_limit = 0.1  # 10% deviation
    BenchmarkWeight = 0.01  # 1% in each bond

    for i in range(N):
        model.addConstr(w[i] >= BenchmarkWeight - deviation_limit * BenchmarkWeight)
        model.addConstr(w[i] <= BenchmarkWeight + deviation_limit * BenchmarkWeight)

    for s in range(S):
        model.addConstr(
            z[s] >= gp.quicksum(w[i] * scenario_returns[s, i] for i in range(N)) - VaR,
            name=f"CVaR_Constraint_{s}"
        )

    # Objective function: Minimize CVaR risk (USING S)
    model.setObjective(
        VaR + (1 / (S * (1 - alpha))) * gp.quicksum(z[s] for s in range(S)),
        GRB.MINIMIZE
    )

    # Optimize the model
    model.optimize()

    if model.status == GRB.OPTIMAL:
        #print("Optimal solution found. List of all weights:")
        weights = [w[i].X for i in range(N)]  # Get the optimized weights for bonds
        return np.array(weights)

    if model.status == GRB.INFEASIBLE:
        print("The model is infeasible.")
        return None
